# Python Tutorial: Miscellaneous

## Lambda Functions
Lambda functions in Python are small anonymous functions defined with the `lambda` keyword. They are used to create small, one-time, throwaway functions without naming them. Lambda functions can take any number of arguments but can have only one expression. The expression is evaluated and returned. The syntax for a lambda function is:

In [45]:
lambda arguments: expression

<function __main__.<lambda>(arguments)>

Imagine you have some brain image, and that you want to threshold that image to keep only the pixels above a certain intensity. You can define a lambda function to do this. First, we make synthetic data:

In [50]:
import numpy as np

# Create a 3D array with dimensions 64x64x64
image_shape = (64, 64, 64)

# Generate random intensities within a range to simulate image data
np.random.seed(42) # For reproducibility
image_data = np.random.randint(0, 255, image_shape)

Next, we define then apply the lambda function to this data.

We will use 2 new functions:

`numpy.zeros_like` is used to create a new array with the same shape and type as a given array, but with all elements set to zero.
 
 `numpy.ndindex` is a function that returns an iterator yielding pairs of indices corresponding to every element in an N-dimensional array. It's particularly useful for looping over the elements of a multi-dimensional array.

In [51]:
threshold_value = 100
threshold_func = lambda x: x if x > threshold_value else 0

thresholded_image = np.zeros_like(image_data)
for ijk in np.ndindex(image_shape):
    thresholded_image[ijk] = threshold_func(image_data[ijk])
print(np.mean(thresholded_image))

107.05646133422852


The `thresholded_image` now contains the thresholded values, keeping only the intensities above 100. 

Another way to do this is to use `numpy.vectorize`. `numpy.vectorize` takes a function that accepts scalar input (a single number) and returns a new function that can handle vector input (arrays). Essentially, it allows you to apply a function designed for individual elements to an entire array at once. It can be much faster than using a for loop:

In [52]:
thresholded_image = np.vectorize(threshold_func)(image_data)
print(np.mean(thresholded_image))

107.05646133422852


## `*args` and `**kwargs`


The `*args` syntax in a function signature is used to pass a variable number of non-keyword arguments to a function. Inside the function, args is a tuple containing all the passed arguments. A tuple is an immutable collection of objects.

In this example, we calculate mean intensity of any number of slices from the synthetic data:

In [53]:
def mean_intensity(*slices):
    total = 0
    count = 0
    for slice in slices:
        total += np.sum(slice)
        count += np.prod(slice.shape)
    return total / count

mean_int = mean_intensity(image_data[0], image_data[1])
print("Mean Intensity:", mean_int)

mean_int = mean_intensity(image_data[10], image_data[30])
print("Mean Intensity:", mean_int)


Mean Intensity: 127.531005859375
Mean Intensity: 126.1837158203125


The `**kwargs` syntax allows you to pass a variable number of keyword arguments to a function. Inside the function, kwargs is a dictionary containing all the keyword arguments. Dictionaries are mutable data structures that allow you to store key-value pairs.

In [54]:
def custom_threshold(image, **kwargs):
    threshold_value = kwargs.get('threshold_value', 100)
    default_value = kwargs.get('default_value', 0)
    return np.where(image > threshold_value, image, default_value)

thresholded_slice = custom_threshold(image_data[12], threshold_value=200, default_value=10)
print(np.mean(thresholded_slice))
thresholded_slice = custom_threshold(image_data[12])
print(np.mean(thresholded_slice))
thresholded_slice = custom_threshold(image_data[12], threshold_value=200)
print(np.mean(thresholded_slice))

56.06982421875
106.9482421875
48.19140625


## Generators and List Comprehension

A generator is a special type of iterator that allows you to iterate over a large sequence of values lazily, meaning one value at a time. Generators are defined using functions that contain one or more `yield` statements.

Imagine you want to iterate over the slices of our synthetic brain data. You can make a generator to yield one slice at a time:

In [56]:
def generate_slices(image_3d):
    depth = image_3d.shape[0]
    for i in range(depth):
        yield image_3d[i, :, :]

for image_slice in generate_slices(image_data):
    # Process each slice here
    custom_threshold(image_slice)

You can use the `next()` function to manually iterate through the values generated by a generator.

In [64]:
slices = generate_slices(image_data)
first_slice = next(slices)
first_thresholded_slice = custom_threshold(first_slice)
print(np.mean(first_thresholded_slice))

108.22802734375


Generator expressions are a more concise way to create generators:

In [65]:
thresholded_slices = (custom_threshold(image_slice) for image_slice in image_data)
first_thresholded_slice = next(thresholded_slices)
print(np.mean(first_thresholded_slice))

108.22802734375


A similar concept to generator expressions is list comprehension. A list comprehension is a one-liner that creates a new list by applying an expression to each item in an existing iterable. The basic syntax of a list comprehension is similar to generator expressions, but using square brackets:


In [66]:
thresholded_slices = [custom_threshold(image_slice) for image_slice in image_data]
first_thresholded_slice = thresholded_slices[0]
print(np.mean(first_thresholded_slice))

108.22802734375


## Basic Exception Handling

In Python, exception handling is done using the `try`, `except`, `else`, and `finally` blocks. As an example, we will try to read a file path that does not exist.

The `try` block contains the code that might cause an exception, and the `except` block contains the code that will execute if an exception occurs:

The `try` block lets you test a block of code for errors. The code inside the try block is executed, and if any exception occurs, the code inside the try block stops running, and control is passed to the except block. The `except` block lets you handle the error. Here is an example:

In [77]:
try:
    img = nib.load("non_existent_file.nii")
except:
        print("File not found")

File not found


It is better form to specify the exception in the `except` block. By specifying the exception type, you can handle different exceptions in different ways. It also won't mask errors that you'd rather know about:

In [80]:
try:
    image_data = np.load("/Does/Not/Exist")
except FileNotFoundError:
    print("File not found.")
except nib.filebasedimages.ImageFileError:
    print("Invalid image file.")

File not found.


The `else` block will execute if no exceptions are raised in the `try` block. The `finally` block will always execute, whether an exception was raised or not. These blocks are optional.

In [81]:
try:
    image_data = np.load("/Does/Not/Exist")
except FileNotFoundError:
    print("File not found.")
except nib.filebasedimages.ImageFileError:
    print("Invalid image file.")
else:
    print("Image loaded successfully.")
finally:
    print("End of attempt to load image.")

File not found.
End of attempt to load image.


## Decorators

A decorator in Python is a higher-order function that takes a function and extends the behavior of that function without explicitly modifying its code. Decorators are often used to encapsulate common functionality that can be applied to multiple functions or methods. Here's a simple example:

In [82]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


The `@my_decorator` syntax is syntactic sugar for `say_hello = my_decorator(say_hello)`.

Decorators can also take arguments, allowing them to be customized.

In [86]:
import time

def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time} seconds to execute.")
        return result
    return wrapper

@timer_decorator
def process_image_vector(image_data):
    np.vectorize(threshold_func)(image_data)

@timer_decorator
def process_image_loop(image_data):
    thresholded_image = np.zeros_like(image_data)
    for ijk in np.ndindex(image_data.shape):
        thresholded_image[ijk] = threshold_func(image_data[ijk])

process_image_loop(image_data)
process_image_vector(image_data)

process_image_loop took 0.11661696434020996 seconds to execute.
process_image_vector took 0.020823001861572266 seconds to execute.


Finally, the decorator itself can also have parameters:

In [89]:
def logger_decorator(level="INFO"):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(f"[{level}] Calling {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

@timer_decorator
@logger_decorator(level="DEBUG")
def process_image_vector(image_data):
    np.vectorize(threshold_func)(image_data)

@timer_decorator
@logger_decorator(level="DEBUG")
def process_image_loop(image_data):
    thresholded_image = np.zeros_like(image_data)
    for ijk in np.ndindex(image_data.shape):
        thresholded_image[ijk] = threshold_func(image_data[ijk])

process_image_loop(image_data)
process_image_vector(image_data)

[DEBUG] Calling process_image_loop
wrapper took 0.1098637580871582 seconds to execute.
[DEBUG] Calling process_image_vector
wrapper took 0.020756006240844727 seconds to execute.


## Duck Typing
"Duck typing" comes from the saying "If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck." In programming, it means that the type of an object is determined by what it can do (methods and properties), not what it is (class hierarchy).

In a statically typed language, you often have to explicitly declare the type of a variable. In contrast, Python's dynamic typing relies on the properties and methods that an object has, rather than its explicit class or type.

Suppose you're working with vectors, and you want to write a function that adds two vectors. With duck typing, you can write a function that adds any objects as long as they behave like vectors:

In [90]:
def add_vectors(v1, v2):
    return [x + y for x, y in zip(v1, v2)]  # the `zip` function aggregates elements from each of the iterables

class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    
    def __iter__(self):
        return iter([self.x, self.y, self.z])

v1 = [1, 2, 3]
v2 = Vector(4, 5, 6)
result = add_vectors(v1, v2)  # Works, even though v1 and v2 are different types


#### Advantages of Duck Typing
 - Flexibility: Functions can operate on multiple types that have the same behavior.
 - Code Reusability: Write more generic and reusable code.
 - Conciseness: No need to write explicit type checks.

#### Disadvantages of Duck Typing
 - Errors at Runtime: Errors related to incorrect types are only caught at runtime, not during compilation.
 - Less Readability: Without explicit type annotations, understanding the expected behavior of objects may be harder, especially in large codebases.

Python introduced "type hints" as a way to combine the benefits of duck typing with optional static type checking. You can annotate the expected types, which can be checked with tools like `mypy`, but these annotations don't affect the runtime behavior.

In [91]:
from typing import List

def add_vectors(v1: List[float], v2: List[float]) -> List[float]:
    return [x + y for x, y in zip(v1, v2)]

## Docstrings

Docstrings are an essential part of writing maintainable and understandable code!!!!

A docstring is a string literal that occurs as the first statement in a module, function, class, or method definition. They provide a way to document your functions, classes, and modules, describing what they do and how they should be used. Here is a simple example:

In [ ]:
def add_vectors(v1, v2):
    """Adds two vectors element-wise and returns the result."""
    return [x + y for x, y in zip(v1, v2)]

For more complex functions or methods, you can use a multi-line docstring that explains in more detail. The PEP 257 guidelines provide a standard way to write these docstrings.

In [93]:
def sum_image(image_path):
    """
    Sum an image from the given path over all dimensions.

    Parameters:
        image_path (str): The path to the image file.

    Returns:
        float: Sum of the image over all of its dimensions.
    """
    img = nib.load(image_path)
    return np.sum(img.get_fdata()).astype(float)

## Inspection

Inspection in Python refers to the ability to examine various details about a code object at runtime. This is possible through various functions provided by the inspect module, which is part of Python's standard library.

The ability to inspect code objects at runtime is an incredibly powerful tool, especially for debugging, profiling, or writing more dynamic code. Here are some useful functions:

#### Examining Source Code
 - inspect.getsource(object): Returns the source code for a class, method, function, or module.
 - inspect.getfile(object): Returns the name of the file in which an object was defined.

#### Getting Information about Functions and Methods
 - inspect.signature(callable): Returns a Signature object that contains information about the function/method signature, including parameters and annotations.
 - inspect.getargspec(function): Returns an ArgSpec object detailing the arguments of a function (note that this function is considered deprecated as of Python 3.8).

#### Working with Stack Frames and Tracebacks
 - inspect.stack(): Returns information about the current stack, such as the function name, line number, and more.
 - inspect.trace(): Similar to stack(), but returns only the stack information for the caller and other code inside the same thread.

And here's an example:

In [94]:
import inspect

def analyze_function(function):
    """This code will print details about the example_function"""
    print("Name:", function.__name__)
    print("File:", inspect.getfile(function))
    print("Source Code:")
    print(inspect.getsource(function))
    print("Signature:", inspect.signature(function))

def example_function(param1, param2):
    """An example function."""
    return param1 + param2

analyze_function(example_function)

Name: example_function
File: /var/folders/rg/wqvgdwpd2jb1b0vpszd5z2w40000gn/T/ipykernel_74992/3266766537.py
Source Code:
def example_function(param1, param2):
    """An example function."""
    return param1 + param2

Signature: (param1, param2)


We can inspect any function we previously defined:

In [98]:
analyze_function(sum_image)
print("")
analyze_function(add_vectors)

Name: sum_image
File: /var/folders/rg/wqvgdwpd2jb1b0vpszd5z2w40000gn/T/ipykernel_74992/2655166295.py
Source Code:
def sum_image(image_path):
    """
    Sum an image from the given path over all dimensions.

    Parameters:
        image_path (str): The path to the image file.

    Returns:
        float: Sum of the image over all of its dimensions.
    """
    img = nib.load(image_path)
    return np.sum(img.get_fdata()).astype(float)

Signature: (image_path)

Name: add_vectors
File: /var/folders/rg/wqvgdwpd2jb1b0vpszd5z2w40000gn/T/ipykernel_74992/3135730222.py
Source Code:
def add_vectors(v1: List[float], v2: List[float]) -> List[float]:
    return [x + y for x, y in zip(v1, v2)]

Signature: (v1: List[float], v2: List[float]) -> List[float]


Note that if you inspect a function that has already been wrapped by decorators, you will be inspecting the wrapping:

In [99]:
analyze_function(process_image_vector)

Name: wrapper
File: /var/folders/rg/wqvgdwpd2jb1b0vpszd5z2w40000gn/T/ipykernel_74992/3662201407.py
Source Code:
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time} seconds to execute.")
        return result

Signature: (*args, **kwargs)


You can even inspect the decorators themselves!

In [100]:
analyze_function(logger_decorator)

Name: logger_decorator
File: /var/folders/rg/wqvgdwpd2jb1b0vpszd5z2w40000gn/T/ipykernel_74992/2352471217.py
Source Code:
def logger_decorator(level="INFO"):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(f"[{level}] Calling {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

Signature: (level='INFO')



*This tutorial was put together with the help of GPT-4.*